In [1]:
!pip install wandb -qU

In [2]:
import wandb

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb.login(key=user_secrets.get_secret("wandb"))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Loading data 
X_train = pd.read_csv("/kaggle/input/30-days-of-ml/train.csv").iloc[:100_000, :]
X_test = pd.read_csv("/kaggle/input/30-days-of-ml/test.csv")
# Preparing data as a tabular matrix
y_train = X_train.target
X_train = X_train.set_index('id').drop('target', axis='columns')
X_test = X_test.set_index('id')
# Pointing out categorical features
categoricals = [item for item in X_train.columns if 'cat' in item]
# Dealing with categorical data using OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X_train[categoricals] = ordinal_encoder.fit_transform(X_train[categoricals])
X_test[categoricals] = ordinal_encoder.transform(X_test[categoricals])


In [6]:
sweep_config = {
    "method": "bayes", 
    "metric": {
      "name": "rmse",
      "goal": "minimize"   
    },
    "parameters": {
        'learning_rate': {"max": 1.0, "min": 0.01},
        'reg_lambda': {"max": 100.0, "min": 1e-9},
        'reg_alpha': {"max": 100.0, "min": 1e-9},
        'subsample': {"max": 1.0, "min": 0.1},
        'colsample_bytree': {"max": 1.0, "min": 0.1},
        'max_depth': {"max": 12, "min": 1},
        'min_child_weight': {"max": 7, "min": 1},
        'gamma': {"max": 1.0, "min": 0.1},
    }
}

In [7]:
sweep_id = wandb.sweep(sweep_config, project="XGBoost-sweeps")

Create sweep with ID: bxb5tlwo
Sweep URL: https://wandb.ai/lmassaron/XGBoost-sweeps/sweeps/bxb5tlwo


In [8]:
def train(X_train=X_train, y_train=y_train):
    
    config_defaults = {
        'learning_rate': 0.1,
        'reg_lambda': 0.01,
        'reg_alpha': 0.01,
        'subsample': 1.0,
        'colsample_bytree': 1.0,
        'max_depth': 3,
        'min_child_weight': 3,
        'gamma': 0.3,
        'n_estimators': 10_000,
        'random_state': 0,
        'early_stopping_rounds': 300,
        'val_size': 0.2,
        'test_size': 0.25,
        
  }
    
    wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
    config = wandb.config
    
    x, x_val, y, y_val = train_test_split(X_train, y_train, random_state=config.random_state,
                                      test_size=config.val_size)
    
    x, x_test, y, y_test = train_test_split(x, y, random_state=config.random_state, test_size=config.test_size)
    
    params = {
        'learning_rate': config.learning_rate,
        'reg_lambda': config.reg_lambda,
        'reg_alpha': config.reg_alpha,
        'subsample': config.subsample,
        'colsample_bytree': config.colsample_bytree,
        'max_depth': config.max_depth,
        'min_child_weight': config.min_child_weight,
        'gamma': config.gamma,
  }
    
    model = XGBRegressor(
        random_state=config.random_state,
        tree_method = "hist", device = "cuda",
        n_estimators=config.n_estimators,
        early_stopping_rounds=config.early_stopping_rounds,
        **params
    )
    
    model.fit(x, y, eval_set=[(x_val, y_val)], verbose=1000)
    preds = model.predict(x_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    
    print(f"rmse: {rmse:.03f}")
    wandb.log({"rmse": rmse})

In [9]:
wandb.agent(sweep_id, train, count=100)

wandb: Agent Starting Run: 74q8kk6j with config:
wandb: 	colsample_bytree: 0.49544935449878647
wandb: 	gamma: 0.9709130200763636
wandb: 	learning_rate: 0.5032884688720275
wandb: 	max_depth: 4
wandb: 	min_child_weight: 7
wandb: 	reg_alpha: 66.21053082189238
wandb: 	reg_lambda: 6.148868010773329
wandb: 	subsample: 0.42327945854803073
wandb: Currently logged in as: lmassaron. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231226_114601-74q8kk6j
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run curious-sweep-1
wandb: ⭐️ View project at https://wandb.ai/lmassaron/XGBoost-sweeps
wandb: 🧹 View sweep at https://wandb.ai/lmassaron/XGBoost-sweeps/sweeps/bxb5tlwo
wandb: 🚀 View run at https://wandb.ai/lmassaron/XGBoost-sweeps/runs/74q8kk6j


[0]	validation_0-rmse:0.74151
[912]	validation_0-rmse:0.72604
rmse: 0.725


[11:46:35] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

wandb:                                                                                
wandb: 
wandb: Run history:
wandb: rmse ▁
wandb: 
wandb: Run summary:
wandb: rmse 0.725
wandb: 
wandb: 🚀 View run curious-sweep-1 at: https://wandb.ai/lmassaron/XGBoost-sweeps/runs/74q8kk6j
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20231226_114601-74q8kk6j/logs
wandb: Agent Starting Run: 3t7333me with config:
wandb: 	colsample_bytree: 0.29685766972214245
wandb: 	gamma: 0.9581895115628544

[0]	validation_0-rmse:0.74187
[1000]	validation_0-rmse:0.72538
[2000]	validation_0-rmse:0.72473
[2297]	validation_0-rmse:0.72474
rmse: 0.725


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: rmse ▁
wandb: 
wandb: Run summary:
wandb: rmse 0.72451
wandb: 
wandb: 🚀 View run fast-sweep-2 at: https://wandb.ai/lmassaron/XGBoost-sweeps/runs/3t7333me
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20231226_114703-3t7333me/logs
wandb: Agent Starting Run: 7vq9y7o3 with config:
wandb: 	colsample_bytree: 0.49498778021435486
wandb: 	gamma: 0.2994967791752171
wandb: 	learning_rate: 0.0952768988082096
wandb: 	max_depth: 4
wandb: 	min_child_weight: 7
wandb: 	reg_alpha: 77.72647038843827
wandb: 	reg_lambda: 8.41643717596583
wandb: 	subsample: 0.1733618852450931
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231226_114759-7vq9y7o3
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run icy-sweep-3
wandb: ⭐️ View project at https:

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb: rmse ▁
wandb: 
wandb: Run summary:
wandb: rmse 0.72189
wandb: 
wandb: 🚀 View run lively-sweep-98 at: https://wandb.ai/lmassaron/XGBoost-sweeps/runs/iyox5hul
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20231226_132119-iyox5hul/logs
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: a9cf612b with config:
wandb: 	colsample_bytree: 0.5015592721681796
wandb: 	gamma: 0.25658607989283544
wandb: 	learning_rate: 0.3950664039315638
wandb: 	max_depth: 1
wandb: 	min_child_weight: 1
wandb: 	reg_alpha: 1.8882905795907257
wandb: 	reg_lambda: 45.14305654989594
wandb: 	subsample: 0.8909037101052745
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231226_132225-a9cf612b
wandb: Run `wandb offline` to turn off syncing.


[0]	validation_0-rmse:0.74197
[1000]	validation_0-rmse:0.72298
[2000]	validation_0-rmse:0.72135
[2559]	validation_0-rmse:0.72099
rmse: 0.721


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: rmse ▁
wandb: 
wandb: Run summary:
wandb: rmse 0.72106
wandb: 
wandb: 🚀 View run youthful-sweep-99 at: https://wandb.ai/lmassaron/XGBoost-sweeps/runs/a9cf612b
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20231226_132225-a9cf612b/logs
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: m58dl0ud with config:
wandb: 	colsample_bytree: 0.705537713012345
wandb: 	gamma: 0.23647243804474405
wandb: 	learning_rate: 0.29342160833013836
wandb: 	max_depth: 1
wandb: 	min_child_weight: 1
wandb: 	reg_alpha: 29.602380055599404
wandb: 	reg_lambda: 88.28382049647601
wandb: 	subsample: 0.8341383156764027
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231226_132332-m58dl0ud
wandb: Run `wandb offline` to turn off syncing

[0]	validation_0-rmse:0.74223
[1000]	validation_0-rmse:0.72383
[2000]	validation_0-rmse:0.72199
[3000]	validation_0-rmse:0.72119
[4000]	validation_0-rmse:0.72064
[5000]	validation_0-rmse:0.72016
[5839]	validation_0-rmse:0.72004
rmse: 0.720


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: rmse ▁
wandb: 
wandb: Run summary:
wandb: rmse 0.72037
wandb: 
wandb: 🚀 View run lucky-sweep-100 at: https://wandb.ai/lmassaron/XGBoost-sweeps/runs/m58dl0ud
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20231226_132332-m58dl0ud/logs
